## Import

In [ ]:
import xlwings as xw
import pandas as pd
import numpy as np
import time
import os
import requests
import json
import warnings

warnings.filterwarnings('ignore')

from urllib import parse

# path setting
if not os.path.isdir('../datasets/KB/'):
    os.mkdir('../datasets/KB/')
    
os.chdir('../datasets/KB/')

* get data

In [30]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
}

#강원도에서 시, 군 단위 법정동코드 얻기
url_get_city_codes = 'https://api.kbland.kr/land-complex/map/siGunGuAreaNameList?시도명=' 
province  = '서울시'
year = 2023
url_get_city_codes += province
city_codes_response = requests.get(url_get_city_codes, headers = headers)
city_codes_body = city_codes_response.json()['dataBody']['data']
city_codes = []
for c in city_codes_body:
    city_codes.append(c['시군구명'])

C:\Windows\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.kbland.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [10]:
province  = '서울특별시'

In [34]:
url_get_dong_list = 'https://api.kbland.kr/land-complex/map/stutDongAreaNameList?시도명=서울시&시군구명='
c = '종로구'
url_get_dong_list += c
dong_list_response = requests.get(url_get_dong_list, headers = headers)
dong_list_body = dong_list_response.json()['dataBody']['data']

C:\Windows\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.kbland.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
for d in dong_list_body:    
    url_get_apt_list = 'https://api.kbland.kr/land-complex/complexComm/hscmList?법정동코드='    
    url_get_apt_list = url_get_apt_list + d['법정동코드']
    apt_list_response = requests.get(url_get_apt_list, headers = headers)
    if apt_list_response.json()['dataBody']['resultCode'] == 33210:
        continue    
    apt_list_body = apt_list_response.json()['dataBody']['data']

In [45]:
for a in apt_list_body:
    apt_id = a['단지기본일련번호']
    url_apt_info = 'https://api.kbland.kr/land-complex/complex/typInfo?단지기본일련번호=' + str(apt_id)
    url_gongsi_price_list = 'https://api.kbland.kr/land-complex/complex/pubLandPriceByDong?단지기본일련번호=' + str(apt_id)

    gongsi_price_list = requests.get(url_gongsi_price_list, headers = headers).json()['dataBody']
    if gongsi_price_list['resultCode'] == 33210: #kb에 공시가 정보 없음.            
        continue                       

    url_gongsi_dong_price_list = 'https://api.kbland.kr/land-complex/complex/dongHoList?단지기본일련번호=' + str(apt_id)